# Final Project , Part 3 - By Nishi Mehta

#                        How Popular Is Trump Amidst Covid-19? America Responds

In [1]:
import numpy as np
import pandas as pd 
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt
import datetime
import bqplot as bq

In [2]:
from __future__ import print_function
from IPython.display import display
import ipywidgets as ipy
from ipywidgets import Layout, Dropdown, Button
from ipywidgets import Image as ImageIpy

In [3]:
popular_file=pd.read_csv('trump-approval-ratings/approval_topline.csv')
poll_file=pd.read_csv('polls/president_polls.csv')
covid_file=pd.read_csv('covid-19-polls-master/covid_concern_polls.csv')
covid_file2=pd.read_csv('covid-19-polls-master/covid_approval_polls_adjusted.csv')

In [4]:
#covid_file2.tail()

In [5]:
#poll_file.tail()

In [6]:
# covid_file=covid_file.head(100)

In [7]:
#covid_file

In [8]:
# poll_file=poll_file.head(1000)

In [9]:
# converting into datetime
covid_file['end_date']=pd.to_datetime(covid_file['end_date'],format='%Y-%m-%d')
covid_file['start_date']=pd.to_datetime(covid_file['start_date'],format='%Y-%m-%d')

In [10]:
covid_file2['enddate']=pd.to_datetime(covid_file2['enddate'],format='%m/%d/%Y')
covid_file2['startdate']=pd.to_datetime(covid_file2['startdate'],format='%m/%d/%Y')

In [11]:
poll_file['end_date']=pd.to_datetime(poll_file['end_date'],format='%m/%d/%y')
poll_file['start_date']=pd.to_datetime(poll_file['start_date'],format='%m/%d/%y')

In [12]:
#type(covid_file['start_date'][0])

In [13]:
# creating a line plot in matplotlib
# ax = plt.figure(figsize=(16,7))
# plt.plot(covid_file.groupby('end_date').mean()['somewhat'],color='red')
# plt.plot(covid_file.groupby('end_date').mean()['not_at_all'], color='g')
# plt.plot(covid_file.groupby('end_date').mean()['very'], color='orange')
# plt.plot(covid_file.groupby('end_date').mean()['not_very'], color='blue')

# plt.ylabel('Estimate %')
# plt.xlabel('Party')
# plt.title('Approval and Disapproval Percentage')

# plt.show()

In [14]:
x=covid_file['end_date'].sort_values().unique()
x = np.array(x, dtype='datetime64[D]')
y=[covid_file.groupby('end_date').mean()['somewhat'], covid_file.groupby('end_date').mean()['not_at_all'], covid_file.groupby('end_date').mean()['very'],covid_file.groupby('end_date').mean()['not_very']  ] 

In [15]:
#y

In [16]:
#x

In [17]:
import datetime as dt # for formatting min/max in our bqplot.DateScale
x_scl = bq.DateScale(min=dt.datetime(2020, 1, 26), max=dt.datetime(2020, 4,15))

In [18]:
#x_sc = bqplot.LinearScale()
y_sc = bq.LinearScale()
x_ax = bq.Axis(scale = x_scl, label='Date')
y_ax = bq.Axis(scale = y_sc, label='% of American Response', orientation='vertical')
col_sc = bq.ColorScale(colors=['Red', 'Green','Orange','Blue'])

In [19]:
#Interaction
#panzoom = bq.interacts.PanZoom( scales={'x':[x_scl], 'y':[y_sc]})

In [20]:
def_tt=bq.Tooltip(fields=['name', 'index', 'color'])

In [21]:
lines = bq.Lines(x=x, y=y,scales={'x': x_scl, 'y': y_sc,'color':col_sc},tooltip=def_tt,display_legend=True,interactions={'click':'select','hover': 'tooltip'},labels=['somewhat','not_at_all','very','not_very'])

In [91]:
fig = bq.Figure(marks=[lines], axes=[x_ax, y_ax],title='How worried Are Americans About the Economy ?')
fig.layout.height = '700px'
fig.layout.width = '1000px'
#display(fig) # if figures are not showing, give this a shot
# display(ipy.VBox([fig]))

In [95]:
## Dropdown

In [23]:
drop_down = ipy.Dropdown(
            options=['ABC News/Washington Post', 'YouGov ', 'Bendixen & Amandi International ', 'SurveyUSA', 'University of New Hampshire',
                    'University of California, Berkeley ','Selzer & Co.','Tulchin Research','Data for Progress'],
            value='ABC News/Washington Post',
            disabled=False
        )

In [ ]:
## Bar chart for Poll votes

In [92]:
drop_down = ipy.Dropdown(
            options=['ABC News/Washington Post', 'YouGov','SurveyUSA', 'University of New Hampshire',
                    'University of California, Berkeley','Selzer & Co.'],
            value='ABC News/Washington Post',
            disabled=False)


x_sc = bq.OrdinalScale()
y_sc = bq.LinearScale()

x_ax = bq.Axis(scale = x_sc,orientation='vertical', grid_lines = 'dashed')
y_ax = bq.Axis(scale = y_sc, label='Vote%', orientation='horizontal')

poll_y_axis=poll_file[poll_file['pollster']== drop_down.value].groupby('answer').mean()['pct'].nlargest(15)
poll_x_axis=poll_y_axis.index.values.tolist()



poll_bar_chart = bq.Bars(x=poll_x_axis,y=poll_y_axis.tolist(), scales={'x':x_sc,'y':y_sc},  
                         orientation='horizontal',colors=['green'],
           interactions = {'click': 'tooltip'}, selected_style={'fill': 'red'})

# Changing interaction to select on click and tooltip on hover
poll_bar_chart.interactions = {'click': 'select', 'hover': 'tooltip'}



poll_fig = bq.Figure(marks=[poll_bar_chart],axes=[y_ax, x_ax], title='Who wins the most votes? poll by %s'% (drop_down.value))
poll_fig.layout.height = '700px'
poll_fig.layout.width = '1000px'

def on_drop_change(event):
    poll_y_axis=poll_file[poll_file['pollster']== str(event["new"])].groupby('answer').mean()['pct'].nlargest(15)
    poll_bar_chart.y = poll_y_axis.tolist()   
    poll_fig.title = 'Who wins the most votes? poll by ' + event['new'] 
    
drop_down.observe(on_drop_change, ['value'])
    


#display(fig) # if figures are not showing, give this a shot
# display(ipy.VBox([drop_down,poll_fig]))

In [25]:
# poll_file[poll_file['pollster']== drop_down.value].groupby('answer').mean()['pct'].nlargest(15)

In [26]:
no_estimate=100.0-(popular_file['approve_estimate']+popular_file['disapprove_estimate'])

In [27]:
popular_file['no_estimate']=no_estimate

In [28]:
popular_file['modeldate'] = pd.to_datetime(popular_file['modeldate'])

In [29]:
# popular_file.head()

In [30]:
# type(popular_file['modeldate'][0])

In [ ]:
## Dropdown

In [31]:
voter_drop_down = ipy.Dropdown(
            options=['All polls','Voters', 'Adults'],
            value='All polls',
            disabled=False
        )

In [ ]:
## Bar chart of All Polls

In [93]:
dt_val = ipy.DatePicker(
    description='Pick a Date',
    disabled=False,
    value = datetime.date(2020, 4, 9)
)

voter_drop_down = ipy.Dropdown(
            options=['All polls','Voters', 'Adults'],
            value='All polls',
            disabled=False
        )


x_sc = bq.OrdinalScale()
y_sc = bq.LinearScale()
col_sc = bq.OrdinalColorScale(domain=[0, 1], colors=['Purple', 'Gray', 'Blue'])


x_data = ['Approve','Cant Say','Disapprove']
y_data = popular_file.loc[(popular_file['modeldate'].dt.date == dt_val.value) & (popular_file['subgroup']==voter_drop_down.value)][['approve_estimate','no_estimate','disapprove_estimate']].values.tolist()

bar_chart = bq.Bars(x=x_data, y=y_data, scales= {'x': x_sc, 'y': y_sc, 'color': col_sc},
                  type='grouped',
                  interactions={'click': 'select'},
                  colors=['Purple', 'Gray', 'teal'],
                 display_legend=True,
                 labels=['Approve','Cant Say','Disapprove'],
                )
                 

ax_x = bq.Axis(scale=x_sc, orientation='horizontal',label='Voting Class', grid_lines = 'none')
ax_y = bq.Axis(scale=y_sc, orientation='vertical',label = 'Percentage of votes', grid_lines = 'none')

bar_chart.interactions = {'click': 'select'}


voters_fig = bq.Figure(marks=[bar_chart], axes=[ax_x, ax_y], legend_location='right',
               title = 'How many Americans (%s) approve Trump on Date: %s ?'  % (str(voter_drop_down.value),str(dt_val.value)) )
voters_fig.layout.width = '800px'


def on_select_change(event):
    y_data=popular_file.loc[(popular_file['modeldate'].dt.date == event["new"]) & (popular_file['subgroup']==voter_drop_down.value)][['approve_estimate','no_estimate','disapprove_estimate']].values.tolist()
    bar_chart.y = y_data   
    voters_fig.title =  'How many Americans (%s) approve Trump on Date: %s ?'  % (str(voter_drop_down.value),str(event['new']))
    
dt_val.observe(on_select_change, ['value'])
    

def on_voter_drop_change(event):
    y_data=popular_file.loc[(popular_file['modeldate'].dt.date == dt_val.value) & (popular_file['subgroup']== str(event["new"])  )][['approve_estimate','no_estimate','disapprove_estimate']].values.tolist()
    bar_chart.y = y_data   
    voters_fig.title =  'How many Americans (%s) approve Trump on Date: %s ?'  % (str(event['new']),str(dt_val.value))


voter_drop_down.observe(on_voter_drop_change, ['value'])
    


#display(fig) # if figures are not showing, give this a shot
# display(ipy.VBox([dt_val,voter_drop_down, voters_fig]))



In [ ]:
## Line Chart For Polls By Party

In [33]:
noestimate=100.0-(covid_file2['approve']+covid_file2['disapprove'])
#noestimate.head()

In [34]:
covid_file2['noestimate']=noestimate

In [35]:
datax=covid_file2['enddate'].sort_values().unique()
datax = np.array(datax, dtype='datetime64[D]')


In [36]:
#covid_file2.groupby(['party','enddate']).mean()['approve']

In [37]:

datay=[covid_file2.groupby(['party','enddate']).mean()['approve'],covid_file2.groupby(['party','enddate']).mean()['noestimate'],covid_file2.groupby(['party','enddate']).mean()['disapprove']]

In [38]:
#datay[0]['all']

In [113]:


x_scale = bq.DateScale()
y_scale = bq.LinearScale()
col_scale = bq.OrdinalColorScale( colors=['Green', 'Gray', 'Orange'])




line_chart = bq.Lines(x=datax, y=[datay[0]['D'], datay[0]['R'], datay[0]['I'] ], scales= {'x': x_scale, 'y': y_scale, 'color': col_scale},
                  
                  interactions={'click': 'select'},
                  colors=['Green', 'Gray', 'Orange'],
                 display_legend=True,
                 labels=['Democrats','Republicans','Independents'],
                )
                 

xax = bq.Axis(scale=x_scale, orientation='horizontal',label='Date', grid_lines = 'none')
yax = bq.Axis(scale=y_scale, orientation='vertical',label = 'Percentage of votes by Party', grid_lines = 'none')

line_chart.interactions = {'click': 'select'}


party_fig = bq.Figure(marks=[line_chart], axes=[xax, yax], legend_location='top-right',
               title = 'How many votes does each party cater to ?'  )
party_fig.layout.width = '600px'
party_fig.layout.padding_y='100px'


    


#display(fig) # if figures are not showing, give this a shot
#display(ipy.VBox([party_fig]))



# Line chart for America's response

In [110]:
display(ipy.HBox([ipy.VBox([fig])]))

As you can see in the chart above, the share of Americans who are either “somewhat” or “very” concerned about infection has risen steadily since the virus began rapidly spreading in the U.S. in March.

But economic concerns, at least at the moment, seem to be worrying more Americans, as the impact of the virus on the economy has been staggering so far. In late March, millions of workers filed for unemployment each week — many times more than previous record highs — and as you can see in the next chart, the share of Americans who are “very” concerned about the economy grew dramatically in the second half of the month, as cities and states began to shut down nonessential businesses.

# Bar chart for Poll votes

In [111]:
display(ipy.HBox([ipy.VBox([drop_down,poll_fig])]))

An updating calculation of the president's approval rating, accounting for each poll's quality, recency, sample size and partisan lean.

# Bar chart of All Polls & Line Chart By Party

In [114]:
display(ipy.HBox([ipy.VBox([dt_val,voter_drop_down, voters_fig]),ipy.VBox([party_fig])]))

Those economic concerns could play a role in the general election, too, as the way President Trump deals with the pandemic might be a factor in voters’ decisions. Below are all the polls we could find that were conducted since early February and asked Americans whether they approve of the way Trump is handling the crisis.We also broke the answers down by party, and as you can see, Republicans largely approve of Trump’s response, while Democrats do not and independents fall somewhere in between.